In [ ]:
# cac retweet

In [1]:
from pyspark.sql import functions

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5,application_1680859770941_0006,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from py4j.protocol import Py4JJavaError
from pyspark import SparkConf, SparkContext
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import explode, col, udf, concat_ws, from_json, lit, array, expr, size
from pyspark.sql.functions import sum as _sum
from pyspark.sql.types import *
import json
import os
import gc
from pyspark.sql.types import BooleanType
# from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# retweets

input_bucket = 's3://cs5344-twitter-project'
original_sdf = spark.read.format('csv').options(header='True').load('s3://cs5344-twitter-project/input/original_tweet_intermediate_full').cache()
conn_sdf = spark.read.format('json').options(header='True').load('s3://cs5344-twitter-project/input/retweeted_tweet_intermediate_full').cache()



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
conn_sdf.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['connected_user_single', 'created_at', 'id_str', 'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'quoted_original_tweet_id_str', 'quoted_original_user_id_str', 'quoted_time', 'retweeted_original_tweet_id_str', 'retweeted_original_user_id_str', 'retweeted_time', 'text', 'text_hashtag', 'user_id_str']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# retweets
# input_bucket = 's3://cs5344-twitter-project'
# original_sdf = spark.read.format('csv').options(header='True').load('s3://cs5344-twitter-project/input/original_tweet_intermediate_full').cache()
# conn_sdf = spark.read.format('json').options(header='True').load('s3://cs5344-twitter-project/input/retweeted_tweet_intermediate_full').cache()
conn_sdf.createOrReplaceTempView("conn_sdf")
retweets_cnt = spark.sql("""
    select user_id_str, count(1)
    from conn_sdf
    group by user_id_str""")
retweets_cnt.createOrReplaceTempView("retweets_cnt")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
retweets_cnt.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------+
|        user_id_str|count(1)|
+-------------------+--------+
|1480459776328814592|       1|
|1158804531909541888|       2|
|           19664256|       3|
|1329851674823958528|       1|
| 806504056889823232|       2|
|          381646586|       1|
|         1869680923|       4|
|1352121768660578304|       6|
|1485143917591142402|       4|
|1434996729678635008|       1|
|1366187324183355395|       2|
|1154564904663130112|       1|
|         3292379858|       1|
|1328308841390497792|       9|
|1479480879542005763|       2|
|         1007465750|       2|
|          285524571|       2|
|1028223590431444993|       1|
|1466835443715129350|       1|
|1439194393492312064|       1|
+-------------------+--------+
only showing top 20 rows